In [1]:
from project_heart.lv import LV
import numpy as np
import pyvista as pv
pv.set_jupyter_backend("pythreejs")

import os
from pathlib import Path

In [2]:
from project_heart.enums import *

In [3]:
filepath = Path("../sample_files/lvhexmesh.vtk")
lv = LV.from_pyvista_read(filepath)
lv.plot(show_edges=True)

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', posit…

In [4]:
lv.identify_surfaces(
  endo_epi_args=dict(threshold=90.0),
  apex_base_args=dict(ab_ql=0.04, ab_qh=0.69),
  aortic_mitral_args=dict(a1=0.4,
                          a2=0.5,
                          a3=0.3,
                          a4=75,
                          a5=130,
                          
                          m1=0.17,
                          m2=0.02,
                          m3=0.07,
                          m4=0.333
                          )
)

In [5]:
1 in lv._nodesets

True

In [6]:
lv.plot("surface", scalars=LV_MESH_DATA.SURFS.value, 
        vnodes=[
          (LV_VIRTUAL_NODES.MITRAL, "green"),
          (LV_VIRTUAL_NODES.AORTIC, "red"),
          (LV_VIRTUAL_NODES.AORTIC_BORDER, "purple"),
          ],
        cmap="tab20_r")

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', posit…

In [7]:
lv.plot("surface", scalars=LV_MESH_DATA.SURFS_DETAILED.value, 
        vnodes=[
          (LV_VIRTUAL_NODES.MITRAL, "green"),
          (LV_VIRTUAL_NODES.AORTIC, "red"),
          (LV_VIRTUAL_NODES.AORTIC_BORDER, "purple"),
          ],
        cmap="tab20_r")

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', posit…

In [8]:
lv.plot("mesh", scalars=LV_MESH_DATA.AB_ENDO_EPI.value, 
        cat_exclude_zero=True,
        cmap="Set2_r")

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', posit…

In [9]:
lv.plot("mesh", scalars=LV_MESH_DATA.EPI_ENDO.value, 
        cmap="Set2_r")

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', posit…

In [10]:
lv.plot("mesh", scalars=LV_MESH_DATA.AM_EPI_ENDO.value, 
        cat_exclude_zero=True,
        cmap="Set2")

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', posit…

Testing point data to cell data transformation:

In [11]:
cellCat = lv.transform_point_data_to_cell_data(LV_MESH_DATA.AM_EPI_ENDO.value)

In [12]:
lv.mesh.cell_data["TESTCAT"] = cellCat

In [13]:
lv.plot("mesh", scalars="TESTCAT", 
        cat_exclude_zero=True,
        cmap="Set2")

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', posit…

Testing nodesets

In [14]:
APEX = lv.get_nodeset(LV_SURFS.APEX_REGION)
BASE = lv.get_nodeset(LV_SURFS.BASE_REGION)
APEX_BASE = np.zeros(lv.mesh.n_points)
APEX_BASE[APEX] = LV_SURFS.APEX_REGION
APEX_BASE[BASE] = LV_SURFS.BASE_REGION

lv.mesh.point_data["APEX_BASE_TEST"] = APEX_BASE
lv.plot(scalars="APEX_BASE_TEST")

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', posit…

In [15]:
endo = lv.get_nodeset(LV_SURFS.ENDO)
epi = lv.get_nodeset(LV_SURFS.EPI)
endo_epi = np.zeros(lv.mesh.n_points)
endo_epi[endo] = LV_SURFS.ENDO
endo_epi[epi] = LV_SURFS.EPI

lv.mesh.point_data["ENDO_EPI_TEST"] = endo_epi
lv.plot(scalars="ENDO_EPI_TEST", cmap="jet")

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', posit…

In [16]:
endo = lv.get_nodeset(LV_SURFS.ENDO)
epi = lv.get_nodeset(LV_SURFS.EPI)
mitral = lv.get_nodeset(LV_SURFS.MITRAL)
aortic = lv.get_nodeset(LV_SURFS.AORTIC)

test_regions = np.zeros(lv.mesh.n_points)
test_regions[endo] = LV_SURFS.ENDO
test_regions[epi] = LV_SURFS.EPI
test_regions[mitral] = LV_SURFS.MITRAL
test_regions[aortic] = LV_SURFS.AORTIC

lv.mesh.point_data["TEST"] = test_regions
lv.plot(scalars="TEST")

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', posit…

In [17]:
endo = lv.get_nodeset(LV_SURFS.ENDO)
epi = lv.get_nodeset(LV_SURFS.EPI)
mitral = np.union1d(lv.get_nodeset(LV_SURFS.EPI_MITRAL), lv.get_nodeset(LV_SURFS.BORDER_MITRAL))
aortic = lv.get_nodeset(LV_SURFS.BORDER_AORTIC)

test_regions = np.zeros(lv.mesh.n_points)
test_regions[endo] = LV_SURFS.ENDO
test_regions[epi] = LV_SURFS.EPI
test_regions[mitral] = LV_SURFS.MITRAL
test_regions[aortic] = LV_SURFS.AORTIC

lv.mesh.point_data["TEST"] = test_regions
lv.plot(scalars="TEST")

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', posit…

Testing Fiber modes (for surface identification)

In [18]:
lv.identify_fibers_region_ids_ldrb(LV_FIBER_MODES.LDRB_1)
# lv.plot(scalars=LV_MESH_DATA.LDRB_1)

(array([2., 2., 2., ..., 0., 0., 0.]), array([2., 2., 2., ..., 0., 0., 0.]))

In [19]:
lv.plot(scalars=LV_MESH_DATA.LDRB_1.value, cmap="hsv")

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', posit…